<a href="https://colab.research.google.com/github/ASi-F/NLPlay-with-Transformers/blob/main/Phase-3%3A%20Text%20Generation/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

In [2]:
import nltk
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/NLPlay/articles.xlsx')

In [5]:
df['Articles'] = df['Articles'].apply(lambda x: x.replace('\n',''))

In [6]:
df['length'] = df['Articles'].apply(len)

In [7]:
from transformers import GPT2LMHeadModel , GPT2Tokenizer

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large') 
model = GPT2LMHeadModel.from_pretrained('gpt2-large' , 
pad_token_id = tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

In [9]:
df['ref'] = df['Articles'].apply(lambda x: tokenizer.decode(tokenizer.encode(x[:2000],return_tensors='pt')[0]))

In [10]:
def gen(x):
  input_ids = tokenizer.encode(x[:100],return_tensors='pt')
  #output = model.generate(input_ids, max_length = 2000, num_beams = 2,no_repeat_ngram_size  = 2,early_stopping = True)
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 100, top_p = 0.9)
  return output

df['gen'] = df['ref'].apply(gen)

In [32]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []

    len1 = len(ref)
    len2 = len(gen)
    if(len1 >= len2):
      ref = ref.split()[:len2]
    else:
      gen = gen[:len1]
      ref = ref.split()
    
    cc = SmoothingFunction()
    score_bleu = corpus_bleu([ref], [gen],  smoothing_function=cc.method7)
    return score_bleu

In [38]:
x = [bleu(df['ref'][i],df['gen'][i]) for i in range(len(df))]

In [39]:
dict_df = {'Scientific': x[0:10], 'Finance': x[10:20], 'Entertainment': x[20:30]}
BLEU_df = pd.DataFrame(dict_df)
BLEU_df.to_csv(r'BLEU_Scores_GPT.csv', index=False)

In [21]:
df['gen'] = [tokenizer.decode(tok[0], skip_special_tokens= True).split() for tok in df['gen']]

In [26]:
df

,Articles,length,ref,gen
0,The dominant sequence transduction models are ...,2106,The dominant sequence transduction models are ...,"[The, dominant, sequence, transduction, models..."
1,The goal of reducing sequential computation al...,2190,The goal of reducing sequential computation al...,"[The, goal, of, reducing, sequential, computat..."
2,We introduce a new language representation mod...,2176,We introduce a new language representation mod...,"[We, introduce, a, new, language, representati..."
3,"In this paper, we improve the fine-tuning base...",2049,"In this paper, we improve the fine-tuning base...","[In, this, paper,, we, improve, the, fine-tuni..."
4,Many modern NLP systems rely on word embedding...,2597,Many modern NLP systems rely on word embedding...,"[Many, modern, NLP, systems, rely, on, word, e..."
5,We propose the Gaussian Error Linear Unit (GEL...,3425,We propose the Gaussian Error Linear Unit (GEL...,"[We, propose, the, Gaussian, Error, Linear, Un..."
6,This work presents a novel objective function ...,2165,This work presents a novel objective function ...,"[This, work, presents, a, novel, objective, fu..."
7,Semantic Textual Similarity (STS) measures the...,2564,Semantic Textual Similarity (STS) measures the...,"[Semantic, Textual, Similarity, (STS), measure..."
8,We present our UWB system for Semantic Textual...,2574,We present our UWB system for Semantic Textual...,"[We, present, our, UWB, system, for, Semantic,..."
9,The recent tremendous success of unsupervised ...,2414,The recent tremendous success of unsupervised ...,"[The, recent, tremendous, success, of, unsuper..."


In [41]:
BLEU_df

,Scientific,Finance,Entertainment
0,0.377180,0.378955,0.376678
1,0.369200,0.379353,0.355276
2,0.367555,0.374732,0.364949
3,0.376463,0.378874,0.336966
4,0.377092,0.379113,0.350771
5,0.375387,0.377377,0.371982
6,0.372731,0.371400,0.344099
7,0.376642,0.375223,0.372137
8,0.352550,0.365299,0.367234
9,0.374939,0.378563,0.349056
